In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v0.interface import vlstm_fw_torch, vlstm_fwbw_cuda
from src.vlstm_fwbw_v0.interface import vlstm_fw_cuda
from vlstm_formulations.vlstm_parallel import vlstm_parallel_fw_torch
# from vlstm_formulations.vlstm_parallel_tiled import vlstm_parallel_tiled
from src.vlstm_fwbw_v0.torch_impl import vlstm_fw_tiled_torch

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v0/build.ninja...
Building extension module vlstm_fwbw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module vlstm_fwbw_v0...


## CUDA vLSTM forward

In [3]:
S = 64 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
# fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[ 0.0000,  0.1000,  0.2000,  0.3000,  0.4000,  0.5000,  0.6000,  0.7000],
           [ 0.8000,  0.9000,  1.0000,  1.1000,  1.2000,  1.3000,  1.4000,  1.5000],
           [ 1.6000,  1.7000,  1.8000,  1.9000,  2.0000,  2.1000,  2.2000,  2.3000],
           [ 2.4000,  2.5000,  2.6000,  2.7000,  2.8000,  2.9000,  3.0000,  3.1000],
           [ 3.2000,  3.3000,  3.4000,  3.5000,  3.6000,  3.7000,  3.8000,  3.9000],
           [ 4.0000,  4.1000,  4.2000,  4.3000,  4.4000,  4.5000,  4.6000,  4.7000],
           [ 4.8000,  4.9000,  5.0000,  5.1000,  5.2000,  5.3000,  5.4000,  5.5000],
           [ 5.6000,  5.7000,  5.8000,  5.9000,  6.0000,  6.1000,  6.2000,  6.3000],
           [ 6.4000,  6.5000,  6.6000,  6.7000,  6.8000,  6.9000,  7.0000,  7.1000],
           [ 7.2000,  7.3000,  7.4000,  7.5000,  7.6000,  7.7000,  7.8000,  7.9000],
           [ 8.0000,  8.1000,  8.2000,  8.3000,  8.4000,  8.5000,  8.6000,  8.7000],
           [ 8.8000,  8.9000,  9.0000,  9.1000,  9.2000,  9.3000,

In [5]:
hs_pt, n_pt, m_pt, _, matD_pt = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)

In [6]:
# result_gt, m, l, n = vlstm_fw_tiled_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs, bq_tile_size=8, bkv_tile_size=8)
# result_gt, result_gt.shape, torch.exp(-m), l, n

In [7]:
torch.nn.functional.logsigmoid(fgs).squeeze()[17] #+ (10.706538)
(fgs).squeeze()[17] #+ (10.706538)

tensor(1., device='cuda:0')

In [8]:
# cuda kernel
hs_cuda, n_cuda, m_cuda, matD_cuda = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs.squeeze(-1), fgate_preact=fgs.squeeze(-1))

before kernel dispatch - float32!
B: 1, NH: 1, S: 64, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
BIdx.y=0, FLTIdx=7: qTileIdx=0, fChunkAccIterIdx=0 (<1), fWCIdx=0, f_acc=-2.192832, fTileColLast=0.000000
STR: BIdx.y=1: qTileIdx=0, fWCIdx=0 (<1), flatThreadIdx=7: fT_acc_res=-4.698925, fTileCol_val=-4.698925, fTileColLastVal=0.000000
BIdx.y=0, FLTIdx=7: qTileIdx=0, fChunkAccIterIdx=1 (<1), fWCIdx=0, f_acc=-2.192832, fTileColLast=0.000000
BIdx.y=0, FLTIdx=7: qTileIdx=1, fChunkAccIterIdx=0 (<3), fWCIdx=0, f_acc=-2.506094, fTileColLast=-4.698925
STR: BIdx.y=1: qTileIdx=1, fWCIdx=0 (<1), flatThreadIdx=7: fT_acc_res=-9.711113, fTileCol_val=-9.711113, fTileColLastVal=-4.698925
BIdx.y=0, FLTIdx=7: qTileIdx=1, fChunkAccIterIdx=1 (<3), fWCIdx=0, f_acc=-2.506094, fTileColLast=-4.698925
BIdx.y=0, FLTIdx=7: qTileIdx=2, fChunkAccIterIdx=0 (<5), fWCIdx=0, f_acc=-2

In [9]:
matD_cuda[:, :, 0:64, :16]

tensor([[[[-9.2466e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-1.2379e+00, -4.2534e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-1.5512e+00, -7.3861e-01, -2.6438e+00,        -inf,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-1.8644e+00, -1.0519e+00, -2.9571e+00,  1.4518e-01,        -inf,        -inf,        -inf,        -inf,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [-2.1777e+00, -1.3651e+00, -3.2704e+00, -1.6808e-01, -1.2087e-01, 

In [10]:
matD_pt[:, :, 0:64, :16]

tensor([[[[-9.2466e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-1.2379e+00, -4.2534e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-1.5512e+00, -7.3861e-01, -2.6438e+00,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-1.8644e+00, -1.0519e+00, -2.9571e+00,  1.4518e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-2.1777e+00, -1.3651e+00, -3.2704e+00, -1.6808e-01, -1.2087e-01, 

In [20]:
(matD_cuda - matD_pt)[:, :, 0:64, -10:]

tensor([[[[        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
      

In [12]:
hs_cuda - hs_pt

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11],
          [-2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11],
          [-1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10],
          [ 2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10],
          [ 2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10],
          [-4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-

In [13]:
n_cuda.unsqueeze(-1) - n_pt

tensor([[[[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-4.7684e-07],
          [-3.5763e-07],
          [-7.1526e-07],
          [-7.1526e-07],
          [-4.7684e-07],
          [-3.5763e-07],
          [-3.5763e-07],
          [-4.7684e-07],
          [-2.3842e-07],
          [-5.0068e-06],
          [-6.1989e-06],
          [-5.9605e-07],
          [ 2.3842e-07],
          [ 0.0000e+00],
          [-3.5763e-07],
          [ 5.9605e-07],
          [ 1.5497e-06],
          [ 1.3113e-06],
          [ 9.5367e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.9073e-06],
          [ 4.5300e-06],
          [ 2.5034e-06],
          [ 1.0729e-06],


In [14]:
m_cuda.unsqueeze(-1) - m_pt

tensor([[[[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [-2.3842e-07],
          [ 0.0000e+00],
          [-1.7881e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-5.9605e-07],
          [-1.3113e-06],
          [-1.9670e-06],
          [-2.1458e-06],
          [-2.3842e-06],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.5497e-06],
          [-1.3113e-06],
          [-5.9605e-08],
          [ 0.0000e+00],
          [-7.1526e-07],
          [-3.5763e-07],
          [ 8.9407e-07],
          [ 1.1921e-06],
          [ 1.4305e-06],
          [ 1.6689e-06],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],


In [15]:
matD_cuda.shape

torch.Size([1, 1, 64, 64])

In [16]:
matD_cuda[:, :, :10, :10]

tensor([[[[-0.9247,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-1.2379, -0.4253,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-1.5512, -0.7386, -2.6438,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-1.8644, -1.0519, -2.9571,  0.1452,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-2.1777, -1.3651, -3.2704, -0.1681, -0.1209,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-2.4910, -1.6784, -3.5836, -0.4813, -0.4341, -0.5797,    -inf,    -inf,  0.0000,  0.0000],
          [-2.8042, -1.9917, -3.8969, -0.7946, -0.7474, -0.8930, -0.6229,    -inf,  0.0000,  0.0000],
          [-3.1175, -2.3049, -4.2102, -1.1079, -1.0607, -1.2062, -0.9361, -0.3284,  0.0000,  0.0000],
          [-3.4308, -2.6182, -4.5234, -1.4211, -1.3739, -1.5195, -1.2494, -0.6416, -1.0745,    -inf],
          [-3.7440, -2.9314, -4.8367, -1.7344, -1.6872, -1.8328, -1.5626, -0.9549,

In [17]:
matD_pt

tensor([[[[-9.2466e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf],
          [-1.2379e+00, -4.2534e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -in

In [18]:
(matD_cuda - matD_pt)[:, :, :32, :10]

tensor([[[[ 0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 1.1921e-07,  1.1921e-07,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 2.3842e-07,  2.3842e-07,  2.3842e-07,  5.9605e-08,  0.0000e+00,         nan,         nan,         nan,         inf,         inf],
          [ 2.3842e-07,  2.3842e-07,  2.3842e-07,  1.1921e-07,  5.9605e-08,  0.0000e+00,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  5.9605e-08,  0.0000e+00, -5.9605e-08,  0.0000e+00,         nan,         inf,         inf],
      

In [19]:
matD_pt[:, :, :10, :10]

tensor([[[[-0.9247,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.2379, -0.4253,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.5512, -0.7386, -2.6438,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.8644, -1.0519, -2.9571,  0.1452,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-2.1777, -1.3651, -3.2704, -0.1681, -0.1209,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-2.4910, -1.6784, -3.5836, -0.4813, -0.4341, -0.5797,    -inf,    -inf,    -inf,    -inf],
          [-2.8042, -1.9917, -3.8969, -0.7946, -0.7474, -0.8930, -0.6229,    -inf,    -inf,    -inf],
          [-3.1175, -2.3049, -4.2102, -1.1079, -1.0607, -1.2062, -0.9361, -0.3284,    -inf,    -inf],
          [-3.4308, -2.6182, -4.5234, -1.4211, -1.3739, -1.5195, -1.2494, -0.6416, -1.0745,    -inf],
          [-3.7440, -2.9314, -4.8367, -1.7344, -1.6872, -1.8328, -1.5626, -0.9549,